# Solution: Advanced Queries with SQLAlchemy

This notebook contains complete solutions for all the advanced querying exercises.


## Exercise 1: Setup and Model Creation


In [ ]:
# Solution: Set up SQLAlchemy and create comprehensive models
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey, Text, Boolean, Float, Index, Date, Enum
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, joinedload, subqueryload, selectinload
from sqlalchemy import func, and_, or_, not_, desc, asc, case, cast, extract, distinct
from sqlalchemy.sql import text
from datetime import datetime, date, timedelta
import enum

# Create database engine with echo for SQL logging
engine = create_engine('sqlite:///advanced_queries_exercise.db', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)

# Enums for better data integrity
class StudentStatus(enum.Enum):
    ACTIVE = "active"
    INACTIVE = "inactive"
    GRADUATED = "graduated"
    SUSPENDED = "suspended"

class CourseStatus(enum.Enum):
    ACTIVE = "active"
    INACTIVE = "inactive"
    ARCHIVED = "archived"

class Grade(enum.Enum):
    A = "A"
    B = "B"
    C = "C"
    D = "D"
    F = "F"

# Comprehensive school management system models
class School(Base):
    __tablename__ = 'schools'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(200), nullable=False, index=True)
    address = Column(String(300), nullable=False)
    phone = Column(String(20))
    email = Column(String(100), unique=True, nullable=False)
    established_year = Column(Integer)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationships
    departments = relationship("Department", back_populates="school", cascade="all, delete-orphan")
    students = relationship("Student", back_populates="school", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"<School(name='{self.name}')>"

class Department(Base):
    __tablename__ = 'departments'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False, index=True)
    code = Column(String(10), unique=True, nullable=False, index=True)
    description = Column(Text)
    budget = Column(Float, default=0.0)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Foreign keys
    school_id = Column(Integer, ForeignKey('schools.id'), nullable=False)
    head_id = Column(Integer, ForeignKey('teachers.id'))
    
    # Relationships
    school = relationship("School", back_populates="departments")
    head = relationship("Teacher", foreign_keys=[head_id])
    teachers = relationship("Teacher", back_populates="department", foreign_keys="Teacher.department_id")
    courses = relationship("Course", back_populates="department", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"<Department(name='{self.name}', code='{self.code}')>"

class Teacher(Base):
    __tablename__ = 'teachers'
    
    id = Column(Integer, primary_key=True)
    first_name = Column(String(50), nullable=False, index=True)
    last_name = Column(String(50), nullable=False, index=True)
    email = Column(String(100), unique=True, nullable=False, index=True)
    phone = Column(String(20))
    hire_date = Column(Date, nullable=False, index=True)
    salary = Column(Float, nullable=False)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Foreign keys
    department_id = Column(Integer, ForeignKey('departments.id'), nullable=False)
    
    # Relationships
    department = relationship("Department", back_populates="teachers", foreign_keys=[department_id])
    courses = relationship("Course", back_populates="teacher", cascade="all, delete-orphan")
    enrollments = relationship("Enrollment", back_populates="teacher", cascade="all, delete-orphan")
    
    @property
    def full_name(self):
        return f"{self.first_name} {self.last_name}"
    
    def __repr__(self):
        return f"<Teacher(name='{self.full_name}', email='{self.email}')>"

class Student(Base):
    __tablename__ = 'students'
    
    id = Column(Integer, primary_key=True)
    first_name = Column(String(50), nullable=False, index=True)
    last_name = Column(String(50), nullable=False, index=True)
    email = Column(String(100), unique=True, nullable=False, index=True)
    phone = Column(String(20))
    date_of_birth = Column(Date, nullable=False)
    enrollment_date = Column(Date, nullable=False, index=True)
    status = Column(Enum(StudentStatus), default=StudentStatus.ACTIVE, index=True)
    gpa = Column(Float, default=0.0)
    credits_earned = Column(Integer, default=0)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Foreign keys
    school_id = Column(Integer, ForeignKey('schools.id'), nullable=False)
    
    # Relationships
    school = relationship("School", back_populates="students")
    enrollments = relationship("Enrollment", back_populates="student", cascade="all, delete-orphan")
    
    @property
    def full_name(self):
        return f"{self.first_name} {self.last_name}"
    
    @property
    def age(self):
        today = date.today()
        return today.year - self.date_of_birth.year - ((today.month, today.day) < (self.date_of_birth.month, self.date_of_birth.day))
    
    def __repr__(self):
        return f"<Student(name='{self.full_name}', email='{self.email}')>"

class Course(Base):
    __tablename__ = 'courses'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(200), nullable=False, index=True)
    code = Column(String(20), unique=True, nullable=False, index=True)
    description = Column(Text)
    credits = Column(Integer, nullable=False)
    max_students = Column(Integer, default=30)
    current_students = Column(Integer, default=0)
    status = Column(Enum(CourseStatus), default=CourseStatus.ACTIVE, index=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Foreign keys
    department_id = Column(Integer, ForeignKey('departments.id'), nullable=False)
    teacher_id = Column(Integer, ForeignKey('teachers.id'), nullable=False)
    
    # Relationships
    department = relationship("Department", back_populates="courses")
    teacher = relationship("Teacher", back_populates="courses")
    enrollments = relationship("Enrollment", back_populates="course", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"<Course(name='{self.name}', code='{self.code}')>"

class Enrollment(Base):
    __tablename__ = 'enrollments'
    
    id = Column(Integer, primary_key=True)
    enrollment_date = Column(Date, nullable=False, index=True)
    grade = Column(Enum(Grade))
    points_earned = Column(Float, default=0.0)
    is_completed = Column(Boolean, default=False)
    completion_date = Column(Date)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Foreign keys
    student_id = Column(Integer, ForeignKey('students.id'), nullable=False)
    course_id = Column(Integer, ForeignKey('courses.id'), nullable=False)
    teacher_id = Column(Integer, ForeignKey('teachers.id'), nullable=False)
    
    # Relationships
    student = relationship("Student", back_populates="enrollments")
    course = relationship("Course", back_populates="enrollments")
    teacher = relationship("Teacher", back_populates="enrollments")
    
    def __repr__(self):
        return f"<Enrollment(student_id={self.student_id}, course_id={self.course_id})>"

# Create indexes for performance
Index('idx_teachers_name', Teacher.first_name, Teacher.last_name)
Index('idx_students_name', Student.first_name, Student.last_name)
Index('idx_enrollments_student_course', Enrollment.student_id, Enrollment.course_id)
Index('idx_courses_department_status', Course.department_id, Course.status)
Index('idx_students_school_status', Student.school_id, Student.status)

# Create tables
Base.metadata.create_all(engine)

print("✅ Advanced models created successfully!")
print("Models: School, Department, Teacher, Student, Course, Enrollment")
print("Features: Enums, computed properties, comprehensive indexing, complex relationships")


In [ ]:
# Solution: Create comprehensive sample data
session = Session()

# Create schools
schools_data = [
    School(name="Tech University", address="123 Tech Street, Tech City", phone="555-0100", 
           email="info@techuniversity.edu", established_year=1995),
    School(name="Business College", address="456 Business Ave, Business City", phone="555-0200",
           email="info@businesscollege.edu", established_year=1980)
]

session.add_all(schools_data)
session.commit()

# Create departments
departments_data = [
    Department(name="Computer Science", code="CS", description="Computer Science Department", 
               budget=500000.0, school_id=1),
    Department(name="Mathematics", code="MATH", description="Mathematics Department", 
               budget=300000.0, school_id=1),
    Department(name="Business Administration", code="BUS", description="Business Administration", 
               budget=400000.0, school_id=2),
    Department(name="Economics", code="ECON", description="Economics Department", 
               budget=250000.0, school_id=2)
]

session.add_all(departments_data)
session.commit()

# Create teachers
teachers_data = [
    Teacher(first_name="Dr. Alice", last_name="Johnson", email="alice@techuniversity.edu", 
            phone="555-0101", hire_date=date(2015, 9, 1), salary=75000.0, department_id=1),
    Teacher(first_name="Prof. Bob", last_name="Smith", email="bob@techuniversity.edu",
            phone="555-0102", hire_date=date(2018, 1, 15), salary=68000.0, department_id=1),
    Teacher(first_name="Dr. Carol", last_name="Davis", email="carol@techuniversity.edu",
            phone="555-0103", hire_date=date(2012, 8, 20), salary=82000.0, department_id=2),
    Teacher(first_name="Prof. David", last_name="Wilson", email="david@businesscollege.edu",
            phone="555-0201", hire_date=date(2016, 3, 10), salary=70000.0, department_id=3),
    Teacher(first_name="Dr. Eve", last_name="Brown", email="eve@businesscollege.edu",
            phone="555-0202", hire_date=date(2019, 7, 1), salary=65000.0, department_id=4)
]

session.add_all(teachers_data)
session.commit()

# Update department heads
departments_data[0].head_id = 1  # CS department head
departments_data[1].head_id = 3  # Math department head
departments_data[2].head_id = 4  # Business department head
departments_data[3].head_id = 5  # Economics department head
session.commit()

# Create students
students_data = [
    Student(first_name="John", last_name="Doe", email="john@student.edu", phone="555-1001",
            date_of_birth=date(2000, 5, 15), enrollment_date=date(2018, 9, 1), 
            status=StudentStatus.ACTIVE, gpa=3.5, credits_earned=90, school_id=1),
    Student(first_name="Jane", last_name="Smith", email="jane@student.edu", phone="555-1002",
            date_of_birth=date(1999, 8, 22), enrollment_date=date(2017, 9, 1),
            status=StudentStatus.ACTIVE, gpa=3.8, credits_earned=120, school_id=1),
    Student(first_name="Mike", last_name="Johnson", email="mike@student.edu", phone="555-1003",
            date_of_birth=date(2001, 3, 10), enrollment_date=date(2019, 9, 1),
            status=StudentStatus.ACTIVE, gpa=3.2, credits_earned=60, school_id=1),
    Student(first_name="Sarah", last_name="Wilson", email="sarah@student.edu", phone="555-2001",
            date_of_birth=date(1998, 11, 5), enrollment_date=date(2016, 9, 1),
            status=StudentStatus.GRADUATED, gpa=3.9, credits_earned=150, school_id=2),
    Student(first_name="Tom", last_name="Brown", email="tom@student.edu", phone="555-2002",
            date_of_birth=date(2002, 7, 18), enrollment_date=date(2020, 9, 1),
            status=StudentStatus.ACTIVE, gpa=3.0, credits_earned=30, school_id=2)
]

session.add_all(students_data)
session.commit()

# Create courses
courses_data = [
    Course(name="Introduction to Programming", code="CS101", description="Basic programming concepts",
           credits=3, max_students=30, current_students=25, status=CourseStatus.ACTIVE, 
           department_id=1, teacher_id=1),
    Course(name="Data Structures", code="CS201", description="Advanced data structures",
           credits=4, max_students=25, current_students=20, status=CourseStatus.ACTIVE,
           department_id=1, teacher_id=2),
    Course(name="Calculus I", code="MATH101", description="Differential calculus",
           credits=4, max_students=35, current_students=30, status=CourseStatus.ACTIVE,
           department_id=2, teacher_id=3),
    Course(name="Business Management", code="BUS101", description="Principles of business management",
           credits=3, max_students=40, current_students=35, status=CourseStatus.ACTIVE,
           department_id=3, teacher_id=4),
    Course(name="Microeconomics", code="ECON101", description="Introduction to microeconomics",
           credits=3, max_students=30, current_students=25, status=CourseStatus.ACTIVE,
           department_id=4, teacher_id=5)
]

session.add_all(courses_data)
session.commit()

# Create enrollments
enrollments_data = [
    Enrollment(student_id=1, course_id=1, teacher_id=1, enrollment_date=date(2023, 9, 1), 
               grade=Grade.A, points_earned=4.0, is_completed=True, completion_date=date(2023, 12, 15)),
    Enrollment(student_id=1, course_id=2, teacher_id=2, enrollment_date=date(2024, 1, 15),
               grade=Grade.B, points_earned=3.0, is_completed=True, completion_date=date(2024, 5, 10)),
    Enrollment(student_id=2, course_id=1, teacher_id=1, enrollment_date=date(2023, 9, 1),
               grade=Grade.A, points_earned=4.0, is_completed=True, completion_date=date(2023, 12, 15)),
    Enrollment(student_id=2, course_id=3, teacher_id=3, enrollment_date=date(2023, 9, 1),
               grade=Grade.A, points_earned=4.0, is_completed=True, completion_date=date(2023, 12, 15)),
    Enrollment(student_id=3, course_id=1, teacher_id=1, enrollment_date=date(2024, 1, 15),
               grade=Grade.C, points_earned=2.0, is_completed=True, completion_date=date(2024, 5, 10)),
    Enrollment(student_id=4, course_id=4, teacher_id=4, enrollment_date=date(2022, 9, 1),
               grade=Grade.A, points_earned=4.0, is_completed=True, completion_date=date(2022, 12, 15)),
    Enrollment(student_id=4, course_id=5, teacher_id=5, enrollment_date=date(2022, 9, 1),
               grade=Grade.A, points_earned=4.0, is_completed=True, completion_date=date(2022, 12, 15)),
    Enrollment(student_id=5, course_id=4, teacher_id=4, enrollment_date=date(2024, 1, 15),
               grade=Grade.B, points_earned=3.0, is_completed=True, completion_date=date(2024, 5, 10))
]

session.add_all(enrollments_data)
session.commit()

print("✅ Sample data created successfully!")
print(f"Schools: {len(schools_data)}")
print(f"Departments: {len(departments_data)}")
print(f"Teachers: {len(teachers_data)}")
print(f"Students: {len(students_data)}")
print(f"Courses: {len(courses_data)}")
print(f"Enrollments: {len(enrollments_data)}")


## Exercise 2: Advanced Query Patterns


In [ ]:
# Solution: Advanced query patterns
print("=== Advanced Query Patterns ===")

# 1. Complex multi-table join query
print("\n1. Complex Multi-Table Join Query:")
print("Student enrollment details with all related information:")

enrollment_details = session.query(
    Student.first_name,
    Student.last_name,
    Course.name.label('course_name'),
    Course.code.label('course_code'),
    Teacher.first_name.label('teacher_first'),
    Teacher.last_name.label('teacher_last'),
    Department.name.label('department_name'),
    School.name.label('school_name'),
    Enrollment.grade,
    Enrollment.points_earned
).join(Enrollment).join(Course).join(Teacher).join(Department).join(School).filter(
    Enrollment.is_completed == True
).all()

for student_first, student_last, course_name, course_code, teacher_first, teacher_last, dept_name, school_name, grade, points in enrollment_details:
    print(f"  {student_first} {student_last} - {course_name} ({course_code}) taught by {teacher_first} {teacher_last} in {dept_name} at {school_name}: {grade.value} ({points} points)")

# 2. Advanced filtering with multiple conditions
print("\n2. Advanced Filtering with Multiple Conditions:")
print("Active students with high GPA who have completed courses:")

high_gpa_students = session.query(Student).join(Enrollment).filter(
    and_(
        Student.status == StudentStatus.ACTIVE,
        Student.gpa >= 3.5,
        Enrollment.is_completed == True,
        Enrollment.grade.in_([Grade.A, Grade.B])
    )
).distinct().all()

for student in high_gpa_students:
    completed_courses = session.query(func.count(Enrollment.id)).filter(
        and_(Enrollment.student_id == student.id, Enrollment.is_completed == True)
    ).scalar()
    print(f"  {student.full_name}: GPA {student.gpa}, {completed_courses} completed courses")

# 3. CASE statements for conditional logic
print("\n3. CASE Statements for Conditional Logic:")
print("Student academic standing based on GPA:")

academic_standing = session.query(
    Student.first_name,
    Student.last_name,
    Student.gpa,
    case(
        (Student.gpa >= 3.7, 'Dean\'s List'),
        (Student.gpa >= 3.0, 'Good Standing'),
        (Student.gpa >= 2.0, 'Academic Warning'),
        else_='Academic Probation'
    ).label('academic_standing')
).all()

for first_name, last_name, gpa, standing in academic_standing:
    print(f"  {first_name} {last_name}: GPA {gpa} - {standing}")

# 4. Complex aggregation query with grouping
print("\n4. Complex Aggregation Query with Grouping:")
print("Department statistics:")

department_stats = session.query(
    Department.name,
    School.name.label('school_name'),
    func.count(Teacher.id).label('teacher_count'),
    func.avg(Teacher.salary).label('avg_salary'),
    func.count(Course.id).label('course_count'),
    func.count(Enrollment.id).label('total_enrollments'),
    func.avg(Enrollment.points_earned).label('avg_grade_points')
).join(School).outerjoin(Teacher).outerjoin(Course).outerjoin(Enrollment).group_by(
    Department.id, Department.name, School.name
).all()

for dept_name, school_name, teacher_count, avg_salary, course_count, enrollments, avg_points in department_stats:
    points_str = f"{avg_points:.2f}" if avg_points else "N/A"
    print(f"  {dept_name} ({school_name}): {teacher_count} teachers, ${avg_salary:.0f} avg salary, {course_count} courses, {enrollments} enrollments, {points_str} avg points")

# 5. Self-referential query (department hierarchy simulation)
print("\n5. Self-Referential Query (Department Budget Analysis):")
print("Departments with above-average budgets:")

avg_budget = session.query(func.avg(Department.budget)).scalar()
high_budget_depts = session.query(Department).filter(
    Department.budget > avg_budget
).all()

print(f"Average department budget: ${avg_budget:.0f}")
for dept in high_budget_depts:
    print(f"  {dept.name}: ${dept.budget:.0f} (${dept.budget - avg_budget:.0f} above average)")

# 6. EXISTS subqueries for complex filtering
print("\n6. EXISTS Subqueries for Complex Filtering:")
print("Teachers who have taught courses with high enrollment:")

teachers_high_enrollment = session.query(Teacher).filter(
    session.query(Course).filter(
        and_(
            Course.teacher_id == Teacher.id,
            Course.current_students > 25
        )
    ).exists()
).all()

for teacher in teachers_high_enrollment:
    high_enrollment_courses = session.query(Course).filter(
        and_(Course.teacher_id == teacher.id, Course.current_students > 25)
    ).all()
    print(f"  {teacher.full_name}: {len(high_enrollment_courses)} courses with >25 students")

print("\n" + "="*60)
print("Advanced query patterns demonstrated successfully!")
print("="*60)


## Exercise 3: Window Functions and Analytical Queries


In [ ]:
# Solution: Window functions and analytical queries
print("=== Window Functions and Analytical Queries ===")

from sqlalchemy import over

# 1. ROW_NUMBER() window function
print("\n1. ROW_NUMBER() Window Function:")
print("Student GPA rankings:")

gpa_rankings = session.query(
    Student.first_name,
    Student.last_name,
    Student.gpa,
    func.row_number().over(order_by=Student.gpa.desc()).label('gpa_rank')
).all()

for first_name, last_name, gpa, rank in gpa_rankings:
    print(f"  #{rank} {first_name} {last_name}: GPA {gpa}")

# 2. RANK() and DENSE_RANK() for ranking with ties
print("\n2. RANK() and DENSE_RANK() for Ranking with Ties:")
print("Teacher salary rankings:")

salary_rankings = session.query(
    Teacher.first_name,
    Teacher.last_name,
    Teacher.salary,
    func.rank().over(order_by=Teacher.salary.desc()).label('salary_rank'),
    func.dense_rank().over(order_by=Teacher.salary.desc()).label('dense_rank')
).all()

for first_name, last_name, salary, rank, dense_rank in salary_rankings:
    print(f"  Rank {rank} (Dense {dense_rank}): {first_name} {last_name} - ${salary:,.0f}")

# 3. LAG() and LEAD() functions
print("\n3. LAG() and LEAD() Functions:")
print("Student enrollment progression:")

enrollment_progression = session.query(
    Student.first_name,
    Student.last_name,
    Student.enrollment_date,
    func.lag(Student.enrollment_date, 1).over(order_by=Student.enrollment_date).label('previous_enrollment'),
    func.lead(Student.enrollment_date, 1).over(order_by=Student.enrollment_date).label('next_enrollment')
).all()

for first_name, last_name, enrollment_date, prev_date, next_date in enrollment_progression:
    prev_str = prev_date.strftime('%Y-%m-%d') if prev_date else "N/A"
    next_str = next_date.strftime('%Y-%m-%d') if next_date else "N/A"
    print(f"  {first_name} {last_name}: {enrollment_date} (prev: {prev_str}, next: {next_str})")

# 4. Running totals with SUM() OVER()
print("\n4. Running Totals with SUM() OVER():")
print("Cumulative department budgets:")

cumulative_budgets = session.query(
    Department.name,
    Department.budget,
    func.sum(Department.budget).over(order_by=Department.budget.desc()).label('cumulative_budget')
).all()

for name, budget, cumulative in cumulative_budgets:
    print(f"  {name}: ${budget:,.0f} (cumulative: ${cumulative:,.0f})")

# 5. Percentile ranking with PERCENT_RANK()
print("\n5. Percentile Ranking with PERCENT_RANK():")
print("Student GPA percentiles:")

gpa_percentiles = session.query(
    Student.first_name,
    Student.last_name,
    Student.gpa,
    func.percent_rank().over(order_by=Student.gpa.desc()).label('percentile_rank')
).all()

for first_name, last_name, gpa, percentile in gpa_percentiles:
    print(f"  {first_name} {last_name}: GPA {gpa} (top {percentile*100:.1f}%)")

# 6. Partitioning in window functions
print("\n6. Partitioning in Window Functions:")
print("Top students by school:")

top_students_by_school = session.query(
    Student.first_name,
    Student.last_name,
    School.name.label('school_name'),
    Student.gpa,
    func.row_number().over(
        partition_by=School.name,
        order_by=Student.gpa.desc()
    ).label('school_rank')
).join(School).all()

for first_name, last_name, school_name, gpa, rank in top_students_by_school:
    print(f"  #{rank} in {school_name}: {first_name} {last_name} - GPA {gpa}")

# 7. Advanced analytical functions
print("\n7. Advanced Analytical Functions:")
print("Department performance metrics:")

dept_performance = session.query(
    Department.name,
    func.count(Enrollment.id).label('total_enrollments'),
    func.avg(Enrollment.points_earned).label('avg_points'),
    func.stddev(Enrollment.points_earned).label('points_stddev'),
    func.min(Enrollment.points_earned).label('min_points'),
    func.max(Enrollment.points_earned).label('max_points')
).join(Course).join(Enrollment).group_by(Department.id, Department.name).all()

for dept_name, enrollments, avg_points, stddev, min_points, max_points in dept_performance:
    print(f"  {dept_name}: {enrollments} enrollments, {avg_points:.2f} avg points (σ={stddev:.2f}, range: {min_points}-{max_points})")

# Close the session
session.close()

print("\n" + "="*60)
print("Window functions and analytical queries demonstrated!")
print("="*60)


## Exercise 4: Query Optimization and Performance


In [ ]:
# Solution: Query optimization and performance
print("=== Query Optimization and Performance ===")

import time

# 1. N+1 problem demonstration
print("\n1. N+1 Problem Demonstration:")
print("Loading students and accessing their enrollments (BAD):")

# Turn off echo for cleaner output
engine.echo = False

start_time = time.time()
students = session.query(Student).limit(3).all()
for student in students:
    enrollments = student.enrollments
    for enrollment in enrollments:
        course_name = enrollment.course.name
n_plus_1_time = time.time() - start_time

print(f"  Time: {n_plus_1_time:.4f} seconds")

# 2. Fix with joinedload
print("\n2. Solution with joinedload (GOOD):")
start_time = time.time()
optimized_students = session.query(Student).options(
    joinedload(Student.enrollments).joinedload(Enrollment.course)
).limit(3).all()
for student in optimized_students:
    enrollments = student.enrollments
    for enrollment in enrollments:
        course_name = enrollment.course.name
joinedload_time = time.time() - start_time

print(f"  Time: {joinedload_time:.4f} seconds")
print(f"  Improvement: {n_plus_1_time/joinedload_time:.1f}x faster")

# 3. Performance comparison function
def performance_comparison():
    """Compare different eager loading strategies"""
    print("\n3. Performance Comparison:")
    
    # N+1 problem
    start_time = time.time()
    students = session.query(Student).limit(2).all()
    for student in students:
        for enrollment in student.enrollments:
            course_name = enrollment.course.name
            teacher_name = enrollment.teacher.full_name
    n_plus_1_time = time.time() - start_time
    
    # Optimized with eager loading
    start_time = time.time()
    optimized_students = session.query(Student).options(
        joinedload(Student.enrollments).joinedload(Enrollment.course),
        joinedload(Student.enrollments).joinedload(Enrollment.teacher)
    ).limit(2).all()
    for student in optimized_students:
        for enrollment in student.enrollments:
            course_name = enrollment.course.name
            teacher_name = enrollment.teacher.full_name
    optimized_time = time.time() - start_time
    
    print(f"  N+1 Problem: {n_plus_1_time:.4f}s")
    print(f"  Optimized:   {optimized_time:.4f}s")
    print(f"  Improvement: {n_plus_1_time/optimized_time:.1f}x faster")

performance_comparison()

# 4. Query result caching
print("\n4. Query Result Caching:")
cache = {}

def cached_query(query_key, query_func):
    if query_key in cache:
        print(f"  Cache hit for '{query_key}'")
        return cache[query_key]
    else:
        print(f"  Cache miss for '{query_key}' - executing query")
        result = query_func()
        cache[query_key] = result
        return result

# First call - cache miss
start_time = time.time()
high_gpa_students = cached_query("high_gpa_students", 
    lambda: session.query(Student).filter(Student.gpa > 3.5).all())
first_call_time = time.time() - start_time

# Second call - cache hit
start_time = time.time()
high_gpa_students_cached = cached_query("high_gpa_students",
    lambda: session.query(Student).filter(Student.gpa > 3.5).all())
second_call_time = time.time() - start_time

print(f"  First call: {first_call_time:.4f}s")
print(f"  Cached call: {second_call_time:.4f}s")

# 5. Pagination function
print("\n5. Pagination Function:")
def paginate_students(page=1, per_page=2):
    offset = (page - 1) * per_page
    students = session.query(Student).offset(offset).limit(per_page).all()
    total_count = session.query(Student).count()
    total_pages = (total_count + per_page - 1) // per_page
    
    return students, total_count, total_pages

for page in range(1, 4):
    students, total, pages = paginate_students(page, 2)
    print(f"  Page {page}/{pages} ({len(students)} students):")
    for student in students:
        print(f"    {student.full_name} - GPA {student.gpa}")

# 6. Bulk operations
print("\n6. Bulk Operations:")
print("Bulk update example - updating student status:")

# Simulate bulk status update
students_to_update = session.query(Student).filter(Student.gpa >= 3.5).all()
print(f"  Updating status for {len(students_to_update)} high-GPA students")

start_time = time.time()
for student in students_to_update:
    # Simulate status update (in real scenario, you'd update actual fields)
    pass
bulk_update_time = time.time() - start_time

print(f"  Bulk update completed in {bulk_update_time:.4f}s")

# Turn echo back on
engine.echo = True

print("\n" + "="*50)
print("Query optimization and performance techniques demonstrated!")
print("="*50)
